# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 11:04AM,250796,10,8775514,"Citieffe, Inc.",Released
1,Nov 15 2022 11:03AM,250795,12,8779390,LF of America Corp.,Released
2,Nov 15 2022 10:51AM,250794,21,633303,"NBTY Global, Inc.",Released
3,Nov 15 2022 10:51AM,250793,21,633149,"NBTY Global, Inc.",Released
4,Nov 15 2022 10:48AM,250792,19,GP11172022,"Granules Pharmaceuticals, Inc.",Released
5,Nov 15 2022 10:45AM,250791,10,0086173364,ISDIN Corporation,Released
6,Nov 15 2022 10:37AM,250789,10,SONSB0002022,"Nextsource Biotechnology, LLC",Released
7,Nov 15 2022 10:37AM,250789,10,SONSB0002021,"Nextsource Biotechnology, LLC",Released
8,Nov 15 2022 10:24AM,250788,19,TR-RN-11162022,"GCH Granules USA, Inc.",Released
9,Nov 15 2022 9:22AM,250785,16,8775527,Sartorius Bioprocess Solutions,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,250792,Released,1
26,250793,Released,1
27,250794,Released,1
28,250795,Released,1
29,250796,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250792,NaN,NaN,1.0
250793,NaN,NaN,1.0
250794,NaN,NaN,1.0
250795,NaN,NaN,1.0
250796,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0.0,0.0,48.0
250682,0.0,0.0,64.0
250703,23.0,1.0,0.0
250754,0.0,3.0,2.0
250755,0.0,0.0,29.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0,0,48
250682,0,0,64
250703,23,1,0
250754,0,3,2
250755,0,0,29


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,0,0,48
1,250682,0,0,64
2,250703,23,1,0
3,250754,0,3,2
4,250755,0,0,29


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,,,48
1,250682,,,64
2,250703,23,1,
3,250754,,3,2
4,250755,,,29


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 11:04AM,250796,10,"Citieffe, Inc."
1,Nov 15 2022 11:03AM,250795,12,LF of America Corp.
2,Nov 15 2022 10:51AM,250794,21,"NBTY Global, Inc."
3,Nov 15 2022 10:51AM,250793,21,"NBTY Global, Inc."
4,Nov 15 2022 10:48AM,250792,19,"Granules Pharmaceuticals, Inc."
5,Nov 15 2022 10:45AM,250791,10,ISDIN Corporation
6,Nov 15 2022 10:37AM,250789,10,"Nextsource Biotechnology, LLC"
8,Nov 15 2022 10:24AM,250788,19,"GCH Granules USA, Inc."
9,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions
11,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 11:04AM,250796,10,"Citieffe, Inc.",,,1
1,Nov 15 2022 11:03AM,250795,12,LF of America Corp.,,,1
2,Nov 15 2022 10:51AM,250794,21,"NBTY Global, Inc.",,,1
3,Nov 15 2022 10:51AM,250793,21,"NBTY Global, Inc.",,,1
4,Nov 15 2022 10:48AM,250792,19,"Granules Pharmaceuticals, Inc.",,,1
5,Nov 15 2022 10:45AM,250791,10,ISDIN Corporation,,,1
6,Nov 15 2022 10:37AM,250789,10,"Nextsource Biotechnology, LLC",,,2
7,Nov 15 2022 10:24AM,250788,19,"GCH Granules USA, Inc.",,,1
8,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,,1,1
9,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 11:04AM,250796,10,"Citieffe, Inc.",1,,
1,Nov 15 2022 11:03AM,250795,12,LF of America Corp.,1,,
2,Nov 15 2022 10:51AM,250794,21,"NBTY Global, Inc.",1,,
3,Nov 15 2022 10:51AM,250793,21,"NBTY Global, Inc.",1,,
4,Nov 15 2022 10:48AM,250792,19,"Granules Pharmaceuticals, Inc.",1,,
5,Nov 15 2022 10:45AM,250791,10,ISDIN Corporation,1,,
6,Nov 15 2022 10:37AM,250789,10,"Nextsource Biotechnology, LLC",2,,
7,Nov 15 2022 10:24AM,250788,19,"GCH Granules USA, Inc.",1,,
8,Nov 15 2022 9:22AM,250785,16,Sartorius Bioprocess Solutions,1,1,
9,Nov 15 2022 9:22AM,250783,16,Sartorius Bioprocess Solutions,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 11:04AM,250796,10,"Citieffe, Inc.",1,,
1,Nov 15 2022 11:03AM,250795,12,LF of America Corp.,1,,
2,Nov 15 2022 10:51AM,250794,21,"NBTY Global, Inc.",1,,
3,Nov 15 2022 10:51AM,250793,21,"NBTY Global, Inc.",1,,
4,Nov 15 2022 10:48AM,250792,19,"Granules Pharmaceuticals, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 11:04AM,250796,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Nov 15 2022 11:03AM,250795,12,LF of America Corp.,1.0,NaN,NaN
2,Nov 15 2022 10:51AM,250794,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Nov 15 2022 10:51AM,250793,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Nov 15 2022 10:48AM,250792,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 11:04AM,250796,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Nov 15 2022 11:03AM,250795,12,LF of America Corp.,1.0,0.0,0.0
2,Nov 15 2022 10:51AM,250794,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Nov 15 2022 10:51AM,250793,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Nov 15 2022 10:48AM,250792,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2256891,184.0,55.0,0.0
12,501573,2.0,0.0,0.0
15,501438,107.0,0.0,0.0
16,752334,3.0,1.0,0.0
19,752283,2.0,1.0,23.0
20,250771,5.0,2.0,2.0
21,752344,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2256891,184.0,55.0,0.0
1,12,501573,2.0,0.0,0.0
2,15,501438,107.0,0.0,0.0
3,16,752334,3.0,1.0,0.0
4,19,752283,2.0,1.0,23.0
5,20,250771,5.0,2.0,2.0
6,21,752344,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,184.0,55.0,0.0
1,12,2.0,0.0,0.0
2,15,107.0,0.0,0.0
3,16,3.0,1.0,0.0
4,19,2.0,1.0,23.0
5,20,5.0,2.0,2.0
6,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,184.0
1,12,Released,2.0
2,15,Released,107.0
3,16,Released,3.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,23.0,2.0,0.0
Executing,55.0,0.0,0.0,1.0,1.0,2.0,0.0
Released,184.0,2.0,107.0,3.0,2.0,5.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,23.0,2.0,0.0
1,Executing,55.0,0.0,0.0,1.0,1.0,2.0,0.0
2,Released,184.0,2.0,107.0,3.0,2.0,5.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,23.0,2.0,0.0
1,Executing,55.0,0.0,0.0,1.0,1.0,2.0,0.0
2,Released,184.0,2.0,107.0,3.0,2.0,5.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()